In [34]:
from selenium import webdriver
from selenium.webdriver.remote.webelement import WebElement
from bs4 import BeautifulSoup
import pickle
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import numpy as np
import sqlite3
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.palettes import viridis
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource,HoverTool,LinearAxis, Range1d

In [2]:
driver = webdriver.Chrome('C:/Users/farazz/Chrome/chromedriver.exe')
driver.get('https://www.ccn.com/news/')


In [6]:
button = driver.find_element(By.XPATH, '//button[text()="Load More Posts"]')
webdriver.ActionChains(driver).move_to_element(button).click(button).perform()

In [12]:
for i in range(10):
    time.sleep(5)
    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "byscripts_ajax_posts_loader_trigger")))
    button.click()
    time.sleep(5)

WebDriverException: Message: unknown error: Element is not clickable at point (127, 1886)
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.30.477700 (0057494ad8732195794a7b32078424f92a5fce41),platform=Windows NT 6.3.9600 x86_64)


In [37]:
button

<selenium.webdriver.remote.webelement.WebElement (session="803b297c65d5906f029942153c73d9ee", element="0.752952380242361-1")>

In [40]:
for i in range(300):
    button = driver.find_element(By.XPATH, '//button[text()="Load More Posts"]')
    l = button.location_once_scrolled_into_view
    webdriver.ActionChains(driver).move_to_element(button).click(button).perform()
    time.sleep(2)

KeyboardInterrupt: 

In [14]:
height = driver.execute_script("return document.body.scrollHeight")

In [19]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

WebDriverException: Message: unknown error: call function result missing 'value'
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.30.477700 (0057494ad8732195794a7b32078424f92a5fce41),platform=Windows NT 6.3.9600 x86_64)


In [41]:
source = driver.page_source

In [42]:
with open('news_html_ccn.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(source, f, pickle.HIGHEST_PROTOCOL)

In [44]:
soup = BeautifulSoup(source,'html.parser')

In [74]:
doc = soup.find("article")
doc.find("a")['title']

'UK Financial Regulator Launches 24 Cryptocurrency Investigations'

In [78]:
news_docs = list()
for doc in soup.find_all("article"):
    news_doc = dict()
    news_doc['title'] = doc.find("a").get('title')
    news_doc['time'] = doc.time.text.strip()
    news_docs.append(news_doc)

In [86]:
news_data_ccn = pd.DataFrame(news_docs)
news_data_ccn.index = pd.to_datetime(news_data_ccn['time'])
news_data_ccn.head()
news_data_ccn = news_data_ccn.drop('time',axis=1)
news_data_ccn.head()

,title
time,
2018-05-30 01:30:00,UK Financial Regulator Launches 24 Cryptocurre...
2018-05-30 00:05:00,Italy’s Economic Pain Is the Bitcoin Price’s Gain
2018-05-29 23:09:00,Average ICO Investor Makes 82 Percent Profit: ...
2018-05-29 22:05:00,Cryptocurrency Trading Halted in Zimbabwe Desp...
2018-05-29 21:08:00,Crypto World ’Definitely in a Bubble’: Wikiped...


In [87]:
news_data_ccn.to_csv('news_data_ccn.csv')

In [54]:
#for coindesk news source
docs = soup.find("div", "post-info")
t = doc.find('a','fade').text
dt = doc.find('p','timeauthor').time.text.split('at')[0]
desc = doc.find('p','desc').text
print('title:{}\ntime:{}\ndesc:{}'.format(t,dt,desc))

title:Kraken CEO: Crypto Exchange Won't Answer New York AG's Inquiry
time:Apr 18, 2018 
desc:An exchange that left New York in 2015 has been contacted by the state's Attorney General. They're not happy about it.


In [157]:
pic_links = list()
for pic in soup.find_all("div","picture"):
    pic_links.append(pic.img['src'])

In [158]:
len(pic_links)

1084

In [173]:
with open('news_images.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(pic_links, f, pickle.HIGHEST_PROTOCOL)

In [159]:
pic_links

['https://media.coindesk.com/uploads/2018/04/Crypto-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/anonymous-man-200x135.jpg',
 'https://media.coindesk.com/uploads/2016/10/bull-wall-street-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/NY-1-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/Schneiderman-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/shutterstock_1068506876-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/Japan-stop-sign-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/01/Bitcoin-on-rail-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/Math-200x135.jpg',
 'https://media.coindesk.com/uploads/2017/09/BTC-and-rupees-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/Data-1-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/shutterstock_775084789-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/barrier-200x135.jpg',
 'https://media.coindesk.com/uploads/2018/04/bitcoin-bullet-200

In [75]:
article_dataset = list()
for article in soup.find_all("div","post-info"):
    article_data = dict()
    article_data['title'] = article.find('a','fade').text
    article_data['time'] = article.find('p','timeauthor').time.text.replace('at','')
    article_data['description'] = article.find('p','desc').text
    article_dataset.append(article_data)

In [76]:
len(article_dataset)

1080

In [119]:
news_data = pd.DataFrame(article_dataset)

In [120]:
news_data.tail()

,description,time,title
1075,"As the year comes to a close, the price of bit...","Dec 28, 2016 16:41",The Price of Bitcoin is Up $150 Over the Last ...
1076,"Vinny Lingham predicts a $1,000 bitcoin by the...","Dec 28, 2016 15:00",Bitcoin in 2017: A Currency Devaluation Hedge ...
1077,"Bitcoin prices surged this week, breaking thro...","Dec 23, 2016 23:14",Bitcoin Prices Rally Nearly 18% As 2016 Draws ...
1078,A look back at the price of ether during some ...,"Dec 23, 2016 21:17",Classic and the DAO: What Drove Ether Prices i...
1079,"Bitcoin prices have shot past $900, the latest...","Dec 23, 2016 14:54","Bitcoin Price Surge Continues, Passes $900 to ..."


In [121]:
news_data['time'] = pd.to_datetime(news_data['time'])

In [122]:
news_data.head()

,description,time,title
0,An exchange that left New York in 2015 has bee...,2018-04-18 19:31:00,Kraken CEO: Crypto Exchange Won't Answer New Y...
1,Has Pornhub's decision to accept payments in v...,2018-04-18 14:40:00,Crypto Privacy Tokens See Price Lift on Pornhu...
2,Bitcoin remains on the hunt for a bullish move...,2018-04-18 10:00:00,Bitcoin Price Defends $8K But Pullback Still i...
3,Some of the cryptocurrency exchanges named tod...,2018-04-17 22:00:00,What Crypto Exchanges Are Saying About New Yor...
4,New York's attorney general is taking a closer...,2018-04-17 17:09:00,"Binance, Bitfinex and More: NY Launches Inquir..."


In [123]:
news_data.set_index('time')

,description,title
time,,
2018-04-18 19:31:00,An exchange that left New York in 2015 has bee...,Kraken CEO: Crypto Exchange Won't Answer New Y...
2018-04-18 14:40:00,Has Pornhub's decision to accept payments in v...,Crypto Privacy Tokens See Price Lift on Pornhu...
2018-04-18 10:00:00,Bitcoin remains on the hunt for a bullish move...,Bitcoin Price Defends $8K But Pullback Still i...
2018-04-17 22:00:00,Some of the cryptocurrency exchanges named tod...,What Crypto Exchanges Are Saying About New Yor...
2018-04-17 17:09:00,New York's attorney general is taking a closer...,"Binance, Bitfinex and More: NY Launches Inquir..."
2018-04-17 15:30:00,Verge's XVG token is fast losing altitude afte...,Sell the News? Verge Token Drops After Porn Pa...
2018-04-17 11:25:00,U.S.-based cryptocurrency exchange Kraken has ...,Cryptocurrency Exchange Kraken to Shutter Serv...
2018-04-17 10:30:00,The technical charts remain bullish for bitcoi...,Support Forming? Why $7.9K Is Bitcoin's New Pr...
2018-04-17 06:00:00,Two economists have developed a model for pric...,Economists Explore Bitcoin's 'Equilibrium Pric...


In [124]:
news_data.index = pd.to_datetime(news_data['time'])

In [125]:
news_data = news_data['2017-01-01':'2018-03-23 00:00:00']

In [126]:
news_data.head()

,description,time,title
time,,,
2018-03-22 14:00:00,EOS has jumped 70 percent in the last few days...,2018-03-22 14:00:00,EOS Eyes Bull Reversal After 70 Percent Spike
2018-03-22 12:59:00,BloXroute's founder worries relay networks tha...,2018-03-22 12:59:00,Better Bitcoin Relay? Crypto VCs Back BloXrout...
2018-03-22 11:10:00,Two South Korean regulators are reportedly lau...,2018-03-22 11:10:00,Korean Regulators to Probe Bank AML Measures f...
2018-03-22 10:35:00,The bulls' failure to beat resistance around $...,2018-03-22 10:35:00,Pullback Ahead? Bitcoin Hits Stiff Resistance ...
2018-03-21 17:50:00,Jack Dorsey has said he believes bitcoin will ...,2018-03-21 17:50:00,Bitcoin Will Be World's 'Single Currency' Says...


In [127]:
news_data = news_data.drop('time',axis=1)

In [128]:
len(news_data)

988

In [89]:
conn = sqlite3.connect('project_data.db')
c = conn.cursor()

In [94]:
news_data.to_sql('news_data',conn,if_exists='replace')

In [95]:
c.execute('select name from sqlite_master')
c.fetchall()

[('exchange_data',),
 ('ix_exchange_data_index',),
 ('crude_oil',),
 ('ix_crude_oil_index',),
 ('gold_price',),
 ('ix_gold_price_index',),
 ('stock_exchange',),
 ('ix_stock_exchange_index',),
 ('blockchain',),
 ('ix_blockchain_index',),
 ('bitcoin_price',),
 ('ix_bitcoin_price_index',),
 ('training_data',),
 ('ix_training_data_Date',),
 ('news_data',),
 ('ix_news_data_time',)]

In [111]:
news_data = news_data[~news_data.index.duplicated(keep='first')]

In [164]:
price = pd.read_sql('select * from bitcoin_price',conn,index_col='index')

In [165]:
news_price = price.join(news_data,how='outer')

In [144]:
news_price.head()

,Price,description,title
2017-01-01 00:00:00,970.26375,NaN,NaN
2017-01-01 01:00:00,970.28725,NaN,NaN
2017-01-01 02:00:00,969.19200,NaN,NaN
2017-01-01 03:00:00,968.91675,NaN,NaN
2017-01-01 04:00:00,967.44650,NaN,NaN


In [168]:
news_price['Date'] = pd.to_datetime(news_price.index)
news_price.head()

,Price,description,title,Date
2017-01-01 00:00:00,970.26375,NaN,NaN,2017-01-01 00:00:00
2017-01-01 01:00:00,970.28725,NaN,NaN,2017-01-01 01:00:00
2017-01-01 02:00:00,969.19200,NaN,NaN,2017-01-01 02:00:00
2017-01-01 03:00:00,968.91675,NaN,NaN,2017-01-01 03:00:00
2017-01-01 04:00:00,967.44650,NaN,NaN,2017-01-01 04:00:00


In [145]:
news_price['Price'] = news_price.fillna(method='ffill')

In [171]:
output_notebook()

Loading BokehJS ...

In [174]:
news_price1 = news_price[~news_price['title'].isnull()]
news_price1.head()

,Price,description,title,Date
2017-01-01 21:04:00,NaN,"The price of bitcoin passed $1,000 during the ...","Bitcoin Price Tops $1,000 in First Day of 2017...",2017-01-01 21:04:00
2017-01-03 12:42:00,NaN,Think Coinbase's IRS issues are about bitcoin ...,Coinbase's IRS Dispute Isn't Just About Bitcoin,2017-01-03 12:42:00
2017-01-04 00:01:00,NaN,Kraken announced 3rd January that it had becom...,Kraken Adds Anonymous Cryptocurrency Monero,2017-01-04 00:01:00
2017-01-04 05:15:00,NaN,The price of bitcoin is now just $110 short of...,Bitcoin Moves Within Striking Distance of All-...,2017-01-04 05:15:00
2017-01-04 18:13:00,NaN,"Bitcoin prices have shot past the $1,100 mark ...",Bitcoin Price Within $25 of New All-Time High,2017-01-04 18:13:00


In [181]:
source1 = ColumnDataSource(news_price)
hover = HoverTool(
    tooltips=[
        ( 'date','@Date'),
        ( 'title','@title' ), 
        ( 'price', '@Price'),
    ],
    formatters={
        'date'      : 'datetime', # use 'datetime' formatter for 'date' field
        'title' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },
    mode='vline')
p = figure(plot_width=800, plot_height=250, x_axis_type="datetime",title='Bitcon price and news',tools=[hover])
p.line('Date', 'Price', color='navy', alpha=0.5,source=source1)

show(p)